In [ ]:
import pandas as pd


In [5]:
import re

df = pd.read_csv('../data/raw/survey.csv')

df_clean = df.replace(to_replace=r"[^\w]+", value="", regex=True)
# mask = True for rows where every column equals the first column’s value
df_clean = df_clean.drop_duplicates(subset=df_clean.columns[1:], keep=False)

# compute mask as above...
# mask = df_clean.astype(str).apply(lambda col: col.str.contains("Milk" "Nothing", case=False, na=False)).any(axis=1)

# # then remove those rows
# df_clean.drop(df_clean[mask].index, inplace=True)
# Mask = True for rows where all values after the 2nd column are identical (case-insensitive)
# Mask = True for rows where all values in columns 3 to 5 (i.e., columns with index 2, 3, 4) are identical (case-insensitive)
def is_english(text):
    # Returns True if text contains only English letters, numbers, spaces, or common punctuation
    if pd.isnull(text):
        return False
    return bool(re.fullmatch(r"[A-Za-z0-9\s,.'\-]+", str(text)))

# Mask for rows where all values in columns 3 to 5 are identical (case-insensitive)
mask_identical = (
    df_clean
    .iloc[:, 1:5]
    .astype(str)
    .apply(lambda row: row.str.lower().nunique() == 1, axis=1)
)

# Mask for rows where all values in columns 3 to 5 are English
mask_english = (
    df_clean
    .iloc[:, 1:5]
    .applymap(is_english)
    .all(axis=1)
)

# Mask for rows where more than two columns contain the substring "Nothing" (case-insensitive)
mask_nothing = (
    df_clean
    .iloc[:, 1:5]
    .astype(str)
    .apply(lambda row: row.str.lower().str.contains("nothing").sum() > 2, axis=1)
)

# Combine all masks: True if any condition is met
mask = mask_identical | ~mask_english | mask_nothing

# Option A: make a filtered copy
df_clean = df_clean[~mask]


df_clean



C:\Users\super\AppData\Local\Temp\ipykernel_30072\2036154768.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(is_english)


,Timestamp,(1) What did you have for breakfast today from the dining hall?,(2) What did you have for lunch from the dining hall? (yesterday),(3) What did you have for dinner from the dining hall? (yesterday),How have you liked the dining hall food so far?
0,78202594855,CerealMilk,PizzaBurger,MongolianBeefandRice,8.0
1,78202594904,yogurt,SearedSalmon,MongolianBeefandRice,5.0
2,78202594918,PotatowedgesNothingyogurtandsausage,MongolianWok,MongolianBeefandRicethebeefwassobad,6.0
5,78202594946,Milk,Nothing,Nothing,2.0
6,78202594948,Potatowedgestheeggsarenotgoodbecausetheyaremad...,CarnitasBurgerSaladSearedSalmonVeganNachos,BurgerPizzaSoupNothing,6.0
7,78202595004,banana,ChickenandriceCarnitasSearedSalmon,MongolianBeefandRicePizzaQuesadilla,8.0
8,78202595006,PotatowedgesOmeletsNothing,ChickenandricePizzaCarnitasBurgerSaladSearedSa...,MongolianBeefandRice,1.0
9,78202595009,Parfeit,SearedSalmonFries,Pizza,7.0
11,78202595013,Parfeit,SearedSalmon,Pizza,7.0
12,78202595046,PotatowedgesOmeletsParfeit,Salad,Salad,6.0


In [ ]:
df.to_csv('../data/cleaned/clean_survey.csv', index=False)